In [1]:
# from typing import TypeVar, Generic, Callable, Union
# from abc import ABC, abstractmethod

# A = TypeVar('A')
# B = TypeVar('B')
# C = TypeVar('C')

In [2]:
# class Either[A, B](ABC):

#     @property
#     @abstractmethod
#     def val(self) -> B: ... # Return the inner value of a Right. Raises an exception if called on a Left.

#     @property
#     @abstractmethod
#     def lval(self) -> A: ... # Return the inner value of a Left. Raises an exception if called on a Right.

#     @abstractmethod
#     def map(self, f: Callable[[B], C]) -> 'Either[A, C]': ...

#     @abstractmethod
#     def flat_map(self, f: Callable[[B], 'Either[A, C]']) -> 'Either[A, C]': ...

# class Left[A](Either):
#     def __init__(self, val: A):
#         self.__val = val

#     @property
#     def is_right(self) -> bool:
#         return False

#     @property
#     def val(self) -> B:
#         raise TypeError('Can\'t return rval from Left()!')

#     @property
#     def lval(self) -> A:
#         return self.__val

#     def map(self, f: Callable[[B], C]) -> Either[A, C]:
#         return self

#     def flat_map(self, f: Callable[[B], Either[A, C]]) -> Either[A, C]:
#         return self

#     def __str__(self) -> str:
#         return f'Left({self.__val})'

# class Right[B](Either):
#     def __init__(self, val: B):
#         self.__val = val

#     @property
#     def is_right(self) -> bool:
#         return True

#     @property
#     def val(self) -> B:
#         return self.__val

#     @property
#     def lval(self) -> A:
#         raise TypeError('Can\'t return lval from Right()!')

#     def map(self, f: Callable[[B], C]) -> Either[A, C]:
#         return Right(f(self.__val))

#     def flat_map(self, f: Callable[[B], Either[A, C]]) -> Either[A, C]:
#         return f(self.__val)

#     def __str__(self) -> str:
#         return f'Right({self.__val})'

In [3]:
from exseos.types.Either import Either, Left, Right

In [4]:
def parse_int(s: str) -> Either[Exception, int]:
	try:
		return Right(int(s))
	except Exception as e:
		return Left(e)

In [5]:
def divide(x: int, y: int) -> Either[Exception, int]:
	if y == 0:
		return Left(ArithmeticError(f"Tried to divide {x} by zero!"))
	else:
		return Right(x // y)

In [6]:
def do_thing(s: str, v: int = 2):
	print(
		parse_int(s)
		.map(lambda x: x**2)
		.flat_map(lambda x: divide(x, v))
		.map(lambda x: f"Wow, it's {x}")
	)

In [7]:
do_thing("4")
do_thing("potato")
do_thing("27", 0)

Right[str](Wow, it's 8)
Left[ValueError](invalid literal for int() with base 10: 'potato')
Left[ArithmeticError](Tried to divide 729 by zero!)


In [8]:
print(parse_int("4").__class__.__name__)
print(None.__class__.__name__)

Right
NoneType


In [9]:
from exseos.types.Result import Result, Okay, Warning, Error

In [10]:
def parse_int_res(s: str) -> Result[Exception, Exception, int]:
	try:
		return Okay(int(s))
	except Exception as e:
		return Error([e])

In [16]:
def optional_div(x: int) -> Result[Exception, Exception, float]:
	try:
		print(f"1/{x} is {1/x}!")
		return Okay(x)
	except Exception:
		return Warning([ArithmeticError(f"Can't calculate 1/{x}")], x)

In [17]:
def cool_op2(s: str):
	x = (
		parse_int_res(s)
		.flat_map(optional_div)
		.map(lambda x: x**2)
		.flat_map(optional_div)
	)

	print(x)

In [20]:
cool_op2("12")
print("\n================")
cool_op2("potato")
print("\n================")
cool_op2("0")

1/12 is 0.08333333333333333!
1/144 is 0.006944444444444444!
Okay[int](144)

Error with no warnings and the following errors:
    > [ValueError] invalid literal for int() with base 10: 'potato'


Warning[int](0) with the following warnings:
    > [ArithmeticError] Can't calculate 1/0
    > [ArithmeticError] Can't calculate 1/0



In [14]:
print(Warning([ArithmeticError("Test")], 1) >> Warning([ArithmeticError("Test")], 2))

Warning[int](2) with the following warnings:
    > [ArithmeticError] Test
    > [ArithmeticError] Test



In [15]:
([ArithmeticError("Test")] if True else []) + (
	[ArithmeticError("Test")] if True else []
)

[ArithmeticError('Test'), ArithmeticError('Test')]